In [3]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 50)
df_churn = pd.read_csv("Data.csv")
df_churn.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,cluster_number
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,84
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No,81
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,56
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,18
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,43


In [4]:
cols = df_churn.columns
df1 = df_churn[['tenure','MonthlyCharges','TotalCharges', 'cluster_number', 'SeniorCitizen']]
df1_int = df1[set(df1.columns)-{'tenure','MonthlyCharges','TotalCharges'}]
df1_float = df1[['tenure','MonthlyCharges','TotalCharges']]
df_churn = df_churn[set(df_churn.columns) - {'tenure','MonthlyCharges','TotalCharges',
                                             'cluster_number', 'SeniorCitizen'}]

def get_item(a):
    return int(a)

def get_fl(a):
    return float(a)

df1_int = df1_int.applymap(get_item)
df1_float = df1_float.applymap(get_fl)
df_churn = df_churn.join(df1_int.join(df1_float))
df_churn.dropna(inplace=True)
df_churn = df_churn[cols]
df_churn.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,cluster_number
0,7590-VHVEG,Female,0,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,84
1,5575-GNVDE,Male,0,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No,81
2,3668-QPYBK,Male,0,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,56
3,7795-CFOCW,Male,0,No,No,45.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,18
4,9237-HQITU,Female,0,No,No,2.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,43


In [6]:
df_churn.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
cluster_number        int64
dtype: object

In [ ]:
df_churn.

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import string
import urllib

url = "https://raw.githubusercontent.com/rujual/telco_churn/master/Data.csv"
file = urllib.request.urlopen(url)
l = []
for line in file:
    decoded_line = line.decode()
    l.append(decoded_line.split(',')[:-1])
df_churn = pd.DataFrame(l[1:], columns=l[0])
empty_cols=['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection','TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

for i in empty_cols:
    df_churn[i]=df_churn[i].replace(" ",np.nan)

df_churn.drop(['customerID'], axis=1, inplace=True)
df_churn = df_churn.dropna()
binary_cols = ['Partner','Dependents','PhoneService','PaperlessBilling']

for i in binary_cols:
    df_churn[i] = df_churn[i].replace({"Yes":1,"No":0})

#Encoding column 'gender'
df_churn['gender'] = df_churn['gender'].replace({"Male":1,"Female":0})


category_cols = ['PaymentMethod','MultipleLines','InternetService','OnlineSecurity',
               'OnlineBackup','DeviceProtection',
               'TechSupport','StreamingTV','StreamingMovies','Contract']

for cc in category_cols:
    dummies = pd.get_dummies(df_churn[cc], drop_first=False)
    dummies = dummies.add_prefix("{}#".format(cc))
    df_churn.drop(cc, axis=1, inplace=True)
    df_churn = df_churn.join(dummies)

df_churn['Churn'] = df_churn['Churn'].replace({"Yes":1,"No":0})


df1 = df_churn.loc[:,:'Churn']
df1_int = df1[set(df1.columns)-{'tenure','MonthlyCharges','TotalCharges'}]
df1_float = df1[['tenure','MonthlyCharges','TotalCharges']]
df2 = df_churn.loc[:,'PaymentMethod#Bank transfer (automatic)':]

def get_item(a):
    return int(a)

def get_fl(a):
    return float(a)

df1_int = df1_int.applymap(get_item)
df1_float = df1_float.applymap(get_fl)
df2 = df2.applymap(get_item)
df_churn = df1_int.join(df1_float.join(df2))
df_churn.dropna(inplace=True)

n_estimators = 100
n_est = n_estimators

y1 = df_churn['Churn']
X1 = df_churn.drop(['Churn'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=0)


sm = SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
X_test_res, y_test_res = sm.fit_sample(X_test, y_test)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [2,4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

import xgboost as xgb

clfxg = xgb.XGBClassifier(objective='binary:logistic', verbosity=0, max_depth=2, eta = 1, silent=0)
clfxg.fit(X_train_res, y_train_res) #, num_round, watchlist)

y_test_pred = clfxg.predict(X_test_res)
conf = confusion_matrix(y_test_res, y_test_pred)
conf

array([[1135,  176],
       [ 189, 1122]])

In [5]:
rfc=RandomForestClassifier(random_state=42,n_estimators=n_est)
gsv_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
rfc.fit(X_train_res, y_train_res)

rfc_best=RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 50, max_depth=8,
                                criterion='gini')

rfc_best.fit(X_train_res, y_train_res)
X_test_res, y_test_res = sm.fit_sample(X_test, y_test)
y_test_pred = rfc_best.predict(X_test_res)
rf_score = rfc_best.score(X_test_res, y_test_res)
conf = confusion_matrix(y_test_res, y_test_pred)
conf

array([[1057,  254],
       [ 146, 1165]])